In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import glob
import matplotlib.pyplot as plt
from scipy.stats import ranksums

In [4]:
import statsmodels
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [10]:
path = '/export/b15/rpapagari/Tianzi_work/ADReSSo_NoVAD_IS2021_dataset/data_ADReSSo_diagnosis_cv10_text_v7_Longformer_TrainDevTest/cv_6/utt2csvpath'
train = '/export/b15/rpapagari/Tianzi_work/ADReSSo_NoVAD_IS2021_dataset/data_ADReSSo_diagnosis_cv10_text_v7_Longformer_TrainDevTest/cv_6/train.tsv'
dev = '/export/b15/rpapagari/Tianzi_work/ADReSSo_NoVAD_IS2021_dataset/data_ADReSSo_diagnosis_cv10_text_v7_Longformer_TrainDevTest/cv_6/dev.tsv'
test = '/export/b15/rpapagari/Tianzi_work/ADReSSo_NoVAD_IS2021_dataset/data_ADReSSo_diagnosis_cv10_text_v7_Longformer_TrainDevTest/cv_6/test.tsv'

In [6]:

def kruskal(f, task, c, p, c_name, p_name):
    for i, title in enumerate(task):
        nome = title
        f.write(('\n'+ f'kruskal results for {title} {c_name} {p_name} {stats.kruskal(c[i], p[i]).pvalue} \n\n'))


def delete_multiple_element(list_object, indices):
    indices = sorted(indices, reverse=True)
    for idx in indices:
        if idx < len(list_object):
            list_object.pop(idx)
            
    return list_object

def read_stats_test(file):
    
    with open(file, 'r') as f:
        lista = []
        testo = f.readlines()
        testo = [line.strip("\n") for line in testo]

        for line in testo:
            if line=="":
                pass
            else:
                lista.append(line)
                
    return lista

def holm_correction(kruskal):
    line_to_remove=[]
    values=[]
    corrected =[]
    final = []
    for l in kruskal:
        if "nan" in l:
            line_to_remove.append(kruskal.index(l))
    
    new_krusk = delete_multiple_element(kruskal, line_to_remove)
            
    for line in new_krusk:
        ok = line.split('vs.')[1]
        num = ok.split(" ")[2]
        values.append(float(num))
   # values = [x for x in values if isnan(x) == False]
    result = statsmodels.stats.multitest.fdrcorrection(values, alpha=0.05, method='indep', is_sorted=False)
    num = np.where(result[0] == True)
    list_index = ((num)[0]).tolist()

    for i in list_index:
        corrected.append(result[1][i])
    for i in list_index:
        final.append(kruskal[i])
    
    return final, corrected
    


def data_to_csv(path_trans, train, dev, test):
    
    path_ordered = []
    sentences = []
    
    read_train = pd.read_csv(train, header=None)
    read_dev = pd.read_csv(dev, header=None)
    read_test = pd.read_csv(test, header=None)
    data = pd.concat([read_train, read_dev, read_test], ignore_index=True)
    
    read_trans = pd.read_csv(path_trans, header=None)
    patients = (data[0].tolist())
    labels = (data[1].tolist())
    path_to_transcript= read_trans[1].tolist()
    
    for patient in patients:
        for path in path_to_transcript:
            if os.path.basename(path).split('.csv')[0] == patient:
                path_ordered.append(path)
                
    for transcript in path_ordered:
        with open(transcript, 'r') as f:
            transcript_ = f.readlines()
        #print(transcript_)
            transcript_ = transcript_[0]
            sentences.append(transcript_)

            
    
    dict = {'idx': patients, 'label': labels, 'sentence': sentences} 
    df = pd.DataFrame(dict)
    return df
    
def compute_best_scores(lista):    

    values = []
    critical = []
    final = [] 
    
    for l in lista:
        ok = l.split('vs.')[1]
        num = ok.split(" ")[2]
        values.append(num)

    for value in values:
        if float(value) < 0.05:
            critical.append(value)

    for li in lista:
        for cri in critical:
            if cri in li:
                final.append(li)

    return final



df = data_to_csv(path, train, dev, test)
len(set(df['idx'].tolist()))
name=df['idx'].tolist()
lab=df['label'].tolist()
dictio = {name_:lab_ for name_, lab_ in zip(name, lab)}


word_analysis = []
path = os.listdir("/export/c12/afavaro/Alignment/Address_2021/csvfiles")
print(len(path))
for elem in path:
    if "word.csv" in elem:
        word_analysis.append(os.path.join('/export/c12/afavaro/Alignment/Address_2021/csvfiles', elem))


base = [os.path.basename(element) for element in word_analysis]
names_ = [ele.split('_PAR')[0] for ele in base]
len(names_)


for element in word_analysis:
    participant = pd.read_csv(element)


list_of_dataframes = []
for element in word_analysis:
    participant = pd.read_csv(element)
    std = participant.Time.std()
    skew = participant.Time.skew()
    d = {'Rhythm_std': std,'rhythm_skew': skew} # 'Rhythm_kurt': kurtosis, 
    df = pd.DataFrame(d, index =[0])
    list_of_dataframes.append(df)

data = pd.concat(list_of_dataframes, sort=False)
data.dropna()
data['name'] = names_


remove =[]
label = []
for r in (data['name']):
    if r in dictio:
        #disorders.append(r)
        label.append(dictio[r])
    else:
        remove.append(r)
       

In [39]:
for e in remove:
    data = data[~data.name.str.contains(e)]


In [40]:
remove =[]
label = []
for r in (data['name']):
    if r in dictio:
        #disorders.append(r)
        label.append(dictio[r])
    else:
        remove.append(r)
       

data['lab'] = label


grouped = data.groupby("lab") #here we extract the values by category
control = grouped.get_group("cn") 
alzheimer =  grouped.get_group("ad")


task = data.columns[0:-2].values.tolist()


task = data.columns[0:-2].values.tolist()
control_all_feat = np.array([control[feat] for feat in np.array(data.columns[0:-2])])
alzhiemer_all_feat = np.array([alzheimer[feat] for feat in np.array(data.columns[0:-2])])



with open('/export/b14/afavaro/Addresso_Results/rhythm.txt', 'w') as f:
    
    kruskal(f, task, control_all_feat, alzhiemer_all_feat, "controls vs.", "alzheimer")


